In [2]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

gtfstk_dir = '../../gtfstk/'
sys.path.append(gtfstk_dir)
import gtfstk as gt

%load_ext autoreload
%autoreload 2

data_dir = gtfstk_dir + 'data/'

In [3]:
filename = 'cairns_gtfs.zip'
#filename = 'portland_gtfs.zip'
feed = gt.read_gtfs(data_dir + filename)
feed.routes

# Set study date
date = gt.get_first_week(feed)[0]
date

'20140526'

In [4]:
trips = feed.trips
trips[trips['trip_id'] == '4526377']
cd = feed.calendar_dates

,route_id,service_id,trip_id,direction_id,block_id,shape_id,trip_type
20244,208,9.387,4526377,0,2833,198553,NaN
